In [1]:
import numpy as np
import rainbow.math.vector3 as V3
import rainbow.math.quaternion as Q
import rainbow.geometry.surface_mesh as MESH
import rainbow.simulators.prox_rigid_bodies.api as API
import rainbow.simulators.prox_rigid_bodies.solver as SOLVER
import rainbow.simulators.prox_rigid_bodies.collision_detection as CD
import rainbow.simulators.prox_rigid_bodies.gauss_seidel as GS
import rainbow.util.viewer as VIEWER
import igl
from rainbow.util.timer import Timer
from third_party.meshplot.meshplot.plot import plot as mplot

# Setup

In [8]:
engine = API.Engine()

engine.params.envelope = 0.3
print('engine.params.envelope = ',engine.params.envelope)

engine.params.resolution = 64
print('engine.params.resolution = ',engine.params.resolution)

API.create_rigid_body(engine,'body_A')
API.create_rigid_body(engine,'body_B')

#V, T = MESH.create_box(5, 6, 4)
V, T = MESH.create_sphere(5, 8, 8)
mesh = API.create_mesh(V, T)
API.create_shape(engine, 'body_a_shape', mesh)

V, T = MESH.create_box(5, 6, 4)
mesh = API.create_mesh(V, T)
API.create_shape(engine, 'body_b_shape', mesh )

API.connect_shape(engine, 'body_A', 'body_a_shape')
API.connect_shape(engine, 'body_B', 'body_b_shape')

API.set_mass_properties(engine,'body_A', 1.0)
API.set_mass_properties(engine,'body_B', 1.0)

API.set_orientation(engine, 'body_A', Q.identity(), use_model_frame=True)
API.set_position(engine, 'body_A', V3.make(10.0,2.0,0.0), use_model_frame=True)
#API.set_position(engine, 'body_A', V3.make(2.0,2.0,0.0), use_model_frame=True)

API.set_orientation(engine, 'body_B', Q.identity(), use_model_frame=True)
API.set_position(engine, 'body_B', V3.make(-10.0, 0.0, 0.0), use_model_frame=True)
#API.set_position(engine, 'body_B', V3.make(-2.0, 0.0, 0.0), use_model_frame=True)


#API.set_orientation(engine, "body_A", Q.Ry(0.3 * np.pi))
API.set_velocity(engine, "body_A", V3.make(-0.1, 0., 0.))
#API.set_spin(engine, "body_A", V3.make(1e-4*np.pi, 0., 0.))

#API.set_orientation(engine, "body_B", Q.Rz(0.3 * np.pi))
API.set_velocity(engine, "body_B", V3.make( 0.01, 0., 0.))
#API.set_spin(engine, "body_B", V3.make(0., 1e-2*np.pi, 0.))


engine.params.envelope =  0.3
engine.params.resolution =  64


# Viewer Functions

In [9]:
def viewer_update_contacts(viewer, engine): 
    K = len(engine.contact_points)
    V = np.zeros((K,3),dtype=np.float64)
    N = np.zeros((K,3),dtype=np.float64)
    for k in range(K):
        V[k,:] = engine.contact_points[k].p
        N[k,:] = engine.contact_points[k].n
    viewer.update_quiver('contacts', V, N, 2.0)

def viewer_update_mesh_position(viewer, engine):
    for body in engine.bodies.values():
        viewer.place_mesh(body.name, body.r, body.q)

# Impulse Based method

In [10]:
# Linear impulse
def compute_impulse(engine, velocity_vector, contact_points):
    '''
        engine         : The physics engine, belongs to the RAINBOW framework
        velocity_vector: The velocity vector contains both linear and angular velocity. 
                         Therefore the vector is of size 6; 3 indexes for the linear velocity 
                         and 3 for the angular velocity. 
        contact_points : List of contact points. Each contact point has the following information. 
                        - Reference to the two objects colliding; objects A and B.
                        - Point of collision
                        - Contact normal
    '''
    body_A              = engine.bodies["body_A"]
    body_B              = engine.bodies["body_B"]
    body_A_mass         = body_A.mass
    body_B_mass         = body_B.mass
    inv_inertia_mass_A  = 1 / body_A.inertia
    inv_inertia_mass_B  = 1 / body_B.inertia
    e                   = 1.0                     #Coefficient of restitution
    
    r_a                 = body_A.r                #Center of mass global space
    r_b                 = body_B.r                #Center of mass global space
    
    body_A_v            = velocity_vector[:3]     #Current Linear Velocity
    body_B_v            = velocity_vector[6:6+3]
    
    body_A_av           = velocity_vector[3:6]    #Current Angular Velocity
    body_B_av           = velocity_vector[6+3:]
    
    for cp in contact_points:
        
        #Compute linear velocity
        v_r = body_A_v - body_B_v           #Relative velocity
        v_j = (- (1 + e) * v_r).dot(cp.n)   #The total velocity of the collision
        
        #Compute Angular velocity
        r_a_cp = r_a - cp.p                 #The world space collision point minus the object’s world space position of body A
        r_b_cp = r_b - cp.p                 #The world space collision point minus the object’s world space position of body B
        
        #Not sure what we do here?
        theta_A = inv_inertia_mass_A * np.cross((np.cross(r_a_cp, cp.n)), r_a_cp)
        theta_B = inv_inertia_mass_B * np.cross((np.cross(r_b_cp, cp.n)), r_b_cp)
        
        # Impulse
        J = (((- (1 + e) * v_r).dot(cp.n)) / 
            ((1/(body_A_mass)) + (1/(body_B_mass)) + (theta_A + theta_B).dot(cp.n)))
        
        #Update velocity vector
        velocity_vector[:3]    += (1/(body_A_mass)) * J * cp.n   
        velocity_vector[6:6+3] -= (1/(body_B_mass)) * J * cp.n
        
        velocity_vector[3:6]   +=  inv_inertia_mass_A * (np.cross(r_a, J*cp.n))
        velocity_vector[6+3:]  -=  inv_inertia_mass_B * (np.cross(r_a, J*cp.n))                            
        
    return velocity_vector

In [11]:
def simulation(engine, viewer, max_iter=100):
    stats = {}
    debug_on = False
    for i in range(max_iter):
        stats = CD.run_collision_detection(engine, stats, debug_on)
        contact_points      = engine.contact_points
        K = len(engine.contact_points)
       
        velocity_vector     = SOLVER.get_velocity_vector(engine)
        position_vector     = SOLVER.get_position_vector(engine)    
        
        velocity_vector     = compute_impulse(engine, velocity_vector, contact_points)
        
        SOLVER.set_velocity_vector(velocity_vector, engine)
        SOLVER.position_update(position_vector, velocity_vector, 1, engine)
        SOLVER.set_position_vector(position_vector, engine)
        
        viewer_update_mesh_position(viewer, engine)
        viewer_update_contacts(viewer, engine)

# Animation

In [12]:
viewer = VIEWER.Viewer()
for body in engine.bodies.values():
    opacity = 0.5
    color = V3.make(1.0,0.1,0.1)
    viewer.create_mesh(body.name, body.shape.mesh.V, body.shape.mesh.T, color, opacity)
viewer.create_quiver('contacts')

viewer_update_mesh_position(viewer, engine)
viewer.show()

Renderer(camera=PerspectiveCamera(aspect=1.25, children=(DirectionalLight(color='white', intensity=0.6, positi…

In [13]:
simulation(engine, viewer, max_iter=1000)